In the same directory, there is `themes.txt` file. That is PDF content provided by Gallup(tm) please note that they are owner of this content.

Countries taken from here: https://gist.github.com/pamelafox/986163 - I had to do some changes, that is why I have a copy.

To run this script pandas, folium, xlrd, xlwt modules are needed.

Have a fun!

Take `themes.txt`, and creates `themes2.xls` with geo coordinates.

In [10]:
from helper import convert_to_excel

#convert_to_excel('themes.txt', 'themes2.xls')

Read dataframe from `themes2.xls`

In [11]:
import pandas as pd
import folium
from semicircle import SemiCircle
from refs import DOMAINS
from helper import legend

df = pd.read_excel('themes2.xls')

remove females, males, all entries (the ones without coordinates)

In [12]:
df = df.dropna(subset=['lon'])
df = df.dropna(subset=['lat'])

Prepare map with stats for Gallup respondents.

In [13]:
sections = [
    {'color': "#007849", 'fill_color':"#015627",
     'description': "Coverage",
     'start_angle':0, 'stop_angle':120},
    {'color': "#8BB0EC", 'fill_color':"#6990BA",
     'description': "Respondents",
     'start_angle':121, 'stop_angle':240},
    {'color': "#FF3C3C", 'fill_color':"#BB0909",
     'description': "Population",
     'start_angle':241, 'stop_angle':360}
]

def put_piechart(folium_map, location, row):
    entries = []
    if row['respondents'] and row['population']:
        cover =  10000000.0 * float(row['respondents']) / float(row['population'])
    else:
        cover = 1
    e = sections[0].copy()
    e['radius'] = cover
    entries.append(e)
    if row['respondents']:
        respondents = float(row['respondents']) / 10.0
    else:
        respondents = 1
    e = sections[1].copy()
    e['radius'] = respondents
    entries.append(e)
    if row['population']:
        population = float(row['population']) / 1000.0
    else:
        population = 1
    e = sections[2].copy()
    e['radius'] = population
    entries.append(e)
    popup_text = f"""country: {row['country']}<br> 
respondants: {respondents}<br> 
population: {population}<br> 
coverage: {cover}
"""

    for e in entries:
        SemiCircle(location=(row['lat'], row['lon']),
                   radius=e['radius'],
                   start_angle=e['start_angle'], stop_angle=e['stop_angle'],
                   color=e['color'], fill_color=e['fill_color'], fill=True, fill_opacity=0.7,
                   popup=popup_text, tooltip=f"{row['country']}"
                  ).add_to(folium_map)


folium_map = folium.Map(location=(32.2297700, 21.0117800), zoom_start=2)

for index, row in df.iterrows():
    put_piechart(folium_map, location=(row["lat"], row["lon"]), row=row)
    
legend(folium_map, sections)

Sample output stored in html - github is not supporting folium....

In [14]:
#folium_map.save("sample_demography.html")

Print out previous map on screen.

In [15]:
folium_map

In [16]:
def calc_sum(row):
    sum = 0
    for d in DOMAINS.values():
        for theme in d['themes']:
            sum += row[theme]
    return sum

def calc_angle(row, domain, sum):
    sub_sum = 0
    for theme in domain['themes']:
        sub_sum += row[theme]

    return 360.0 * sub_sum / sum
        
def put_piechart(folium_map, location, row):
    entries = []
    if row['respondents']:
        respondents = float(row['respondents']) / 10.0
    else:
        respondents = 1
    if respondents < 100000:
        respondents = 100000
    angle = 0
    popup_text = f"""country: {row['country']}<br> 
respondants: {row['respondents']}<br> 
population: {row['population']}
"""
    sum = calc_sum(row)
    for d in DOMAINS.values():
        start_angle = angle
        angle += calc_angle(row, d, sum)
        stop_angle = angle
        #print(f"{row['country']} - {start_angle} - {stop_angle}")
        SemiCircle(location=(row['lat'], row['lon']), 
                   radius=respondents, 
                   start_angle=start_angle, stop_angle=stop_angle,
                   color=d['color'], fill_color=d['color'], fill=True, fill_opacity=0.7,
                   popup=popup_text, tooltip=f"{row['country']}"
                  ).add_to(folium_map)


folium_map = folium.Map(location=(32.2297700, 21.0117800), zoom_start=2)

for index, row in df.iterrows():
    put_piechart(folium_map, location=(row["lat"], row["lon"]), row=row)
    
legend(folium_map, DOMAINS.values())

NameError: name 'plugins' is not defined

Sample output stored in html - github is not supporting folium....

In [13]:
# folium_map.save("sample_themes.html")

Print out map with themes.

In [14]:
folium_map